# Note
For continous training only run the "for cont'" cell

# Preparation

In [1]:
import os

MAIN_PATH = os.getcwd()
TRAINING_PATH = os.path.join(MAIN_PATH, "training")
DATA_PATH = os.path.join(MAIN_PATH, "data")
CONFIG_PATH = os.path.join(DATA_PATH, "pipeline.config")
LABEL_MAP_PATH = os.path.join(DATA_PATH, "label_map.pbtxt")
TRAIN_DATA_PATH = os.path.join(DATA_PATH, "train.record")
VAL_DATA_PATH = os.path.join(DATA_PATH, "validation.record")
TEST_DATA_PATH = os.path.join(DATA_PATH, "test.record")

print(TRAIN_DATA_PATH)

c:\Users\ahmad\OneDrive\College\Bangkit\Capstone V2\train1\data\train.record


## Download the dataset

In [ ]:
import requests

urls = ['https://dl.dropboxusercontent.com/s/9f5suk5oo5u7yyi/test.record',
        'https://dl.dropboxusercontent.com/s/tqvtgswib6r4f8u/validation.record',
        'https://dl.dropboxusercontent.com/s/6kgr2pwi7kgtmwb/train.record']
names = ["test.record",
        "validation.record",
        "train.record"]

for url, name in zip(urls, names):
    r = requests.get(url, allow_redirects=True)
    open(os.path.join(DATA_PATH, name), 'wb').write(r.content)

## Clone the TensorFlow Model Garden

In [ ]:
# (For cont')
%cd {MAIN_PATH}
!git clone https://github.com/tensorflow/models.git


## TF Object Detection API

### Compile/install the TF Object Detection API

CD to the models/research

In [23]:
# (For cont')
%cd {MAIN_PATH}/models/research

c:\Users\ahmad\OneDrive\College\Bangkit\Capstone V2\train1\models\research


### Install the protoc files

In [ ]:
!apt install protobuf-compiler -y

In [18]:
# (For cont')

!protoc object_detection/protos/*.proto --python_out=.

Invalid file name pattern or missing input file "./object_detection/protos/*.proto"


(For cont') \
\
To avoid DNN library not found problem, open the object_detection/packages/tf2/setup.py and make sure to add these lines or you can just run the next cell
```
    'tensorflow==2.8.0',
    'tf-models-official==2.8.0',
    'tensorflow_io==0.23.1',
    'keras==2.8.0',
```



In [ ]:
#!pip install tensorflow==2.8.0
#!pip install tf-models-official==2.8.0
#!pip install tensorflow_io==0.23.1
#!pip install keras==2.8.0
!pip install tf-models-official
!pip install tensorflow_io
!pip install keras

In [49]:
# (For cont')

!cp object_detection/packages/tf2/setup.py .
!python -m pip install  .

### Test the TF Object Detection installation API

In [ ]:
# (For cont')

!python object_detection/builders/model_builder_tf2_test.py

## Download pre-trained model checkpoint

In [ ]:
%cd {DATA_PATH}

In [ ]:
!apt install wget -y

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
!tar -xzvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

## Modify the pipeline.config (IMPORTANT!)
change the checkpoint, label map, train, and test in data/pipeline.config to below path 

In [9]:
print("Checkpoint: " + os.path.join(DATA_PATH, "ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8", "checkpoint", "ckpt-0"))
print("Label map: " + LABEL_MAP_PATH)
print("Train: " + TRAIN_DATA_PATH)
print("Test: " + TEST_DATA_PATH)


Checkpoint: c:\Users\ahmad\OneDrive\College\Bangkit\Capstone V2\train1\data\ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8\ckpt-0
Label map: c:\Users\ahmad\OneDrive\College\Bangkit\Capstone V2\train1\data\label_map.pbtxt
Train: c:\Users\ahmad\OneDrive\College\Bangkit\Capstone V2\train1\data\train.record
Test: c:\Users\ahmad\OneDrive\College\Bangkit\Capstone V2\train1\data\test.record


Also change add "max_to_keep=None" as parameter in tf.train.Saver() in /research/object_detection/legacy/trainer.py \
So it'll be:\
\
saver = tf.train.Saver(max_to_keep=None, \
keep_checkpoint_every_n_hours=keep_checkpoint_every_n_hours)


## Load Tensorboard

In [ ]:
# (For cont')

%load_ext tensorboard
%tensorboard --logdir {TRAINING_PATH}

# Train The Model

## Fixing incompatible libraries

In [56]:
# (For cont')
#==4.5.4.60
!pip uninstall opencv-python --y
!pip uninstall opencv-contrib-python --y
!pip uninstall opencv-python-headless --y
!pip install opencv-python==4.5.5.64 
!pip install opencv-contrib-python==4.5.5.64 
!pip install opencv-python-headless==4.5.5.64 

Found existing installation: opencv-python 4.5.5.64
Uninstalling opencv-python-4.5.5.64:
  Successfully uninstalled opencv-python-4.5.5.64
Found existing installation: opencv-contrib-python 4.5.5.64
Uninstalling opencv-contrib-python-4.5.5.64:
  Successfully uninstalled opencv-contrib-python-4.5.5.64
Found existing installation: opencv-python-headless 4.5.5.64
Uninstalling opencv-python-headless-4.5.5.64:
  Successfully uninstalled opencv-python-headless-4.5.5.64
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached opencv_python-4.5.5.64-cp36-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (60.5 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
Looking in indexes: https://pypi.org/simple, https:

## Training

In [ ]:
# (For cont')

%cd {MAIN_PATH}/models/research/object_detection

!python model_main_tf2.py \
--pipeline_config_path={CONFIG_PATH} \
--model_dir={TRAINING_PATH} \
--alsologtostderr

## Validation

In [ ]:
!python model_main_tf2.py \
--model_dir={TRAINING_PATH} \
--pipeline_config_path={CONFIG_PATH} \
--checkpoint_dir={TRAINING_PATH}

## Inference/Save The Model

In [ ]:
!python exporter_main_v2.py \
--trained_checkpoint_dir={TRAINING_PATH} \
--pipeline_config_path={CONFIG_PATH} \
--output_directory {DATA_PATH}/inference_graph